In [1]:
import json
import numpy as np
import os
import re
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.util import Cm, Pt, Inches
from pptx.enum.text import MSO_ANCHOR, MSO_AUTO_SIZE
from h2ogpte import H2OGPTE
from mediawikiapi import MediaWikiAPI
from tqdm import tqdm
with open('secrets.txt') as f:
    api = f.read()
    f.close()



In [2]:
client = H2OGPTE(
    address="https://h2ogpte.genai.h2o.ai",
    api_key=api
)

user_query = 'Create a presentation on barbie and oppenheimer (barbenheimer)'

In [3]:
def trim(lst, retain=30):
    '''
    Trims a list. This function was originally used to permute before trimming, but\
    now that functionality is removed, so it appears rather redundant.
    '''
    _ = lst.copy()
    
    return _[0:retain]



def try_and_parse(user_query, function, failed=0, markdown=False):
    '''
    Accepts a function and user_query, an input. Evaluates function(user_query) and 
    converts string output (usually a reply from an llm) into a json value. Use markdown=True
    if the json value is contained within a code chunk.
    '''
    chosen = function(user_query)
    try:
        if not markdown:
            topics = json.loads(chosen.content)
        else:
            print(chosen.content)
            pattern = r'^```(?:\w+)?\s*\n(.*?)(?=^```)```'
            result = re.findall(pattern, chosen.content, re.DOTALL | re.MULTILINE)[0].strip() 
            #print(result)
            topics = json.loads(result)
            
        return topics
    except Exception as e:
        failed+=1
        print(failed)
        print(e)# CHANGE TO LOGGING STATEMENT
        try_and_parse(user_query, function, failed=failed)


## Step 1. What comes to mind when you think about xyz?

In [4]:
search = lambda user_query: client.answer_question(
    question=user_query,
    system_prompt="""You are an assistant whose task is to perform Wikipedia searches on a specific topic.\
    The user is interested to create a presentation about a topic of interest.\
    Reply with at least one corresponding Wikipedia query as an array in JSON format.\
    Only reply with the JSON array and nothing else. Here are some examples.
    Example 1.
    User: Create a presentation on the book, Baby Rudin.
    Assistant: ["Real analysis", "Mathematical Analysis"]

    Example 2.
    User: I want to create a ppt about Milk.
    Assistant: ["Milk", "Plant Milk", "Animal Milk", "Almond Milk"]
    """,
    llm='mistralai/Mixtral-8x7B-Instruct-v0.1' # i like this model
)
searched = try_and_parse(user_query, search)
searched

['Barbie',
 'Oppenheimer',
 'Barbie and Oppenheimer in popular culture',
 'Barbie (doll)',
 'J. Robert Oppenheimer']

## Step 2. Search Wikipedia

In [5]:
import random
wiki = MediaWikiAPI()

articles = list(
    set(
        
        [i for j in [wiki.search(cat, results=5) for cat in searched] for i in j]
        
    )
) # remove duplicates with set(list())
random.shuffle(articles) # random shuffle



In [6]:
snippet = trim(
    list(map(lambda x: wiki.summary(x, auto_suggest=False, sentences=1), articles))
)

snippet_ = snippet.copy()
i = 0
for string in snippet:
    string = f"{i}. {string}"
    snippet[i] = string
    i+=1
snippet_text = "\n\n".join(snippet)
print(snippet_text)

0. Kenneth Sean "Ken" Carson Jr.

1. Margaret "Midge" Hadley Sherwood is a doll character in the Barbie line of toys by Mattel that was first released in 1963.

2. Barbie is a fashion doll created by American businesswoman Ruth Handler, manufactured by American toy and entertainment company Mattel and introduced on March 9, 1959.

3. Nikolaus Barbie (25 October 1913 – 25 September 1991) was a German officer of the SS and SD who worked in Vichy France during World War II. He became known as the "Butcher of Lyon" for having personally tortured prisoners—primarily Jews and members of the French Resistance—as the head of the Gestapo in Lyon.

4. Go woke, go broke, or alternatively get woke, go broke, is an American political catchphrase used by some political pundits to refer to the actual or perceived stock value drops or loss in sales ("going broke") of companies or corporations that publicly support progressive causes, such as the treatment of women, LGBT people and people of color (ter

## Step 3. "Brainstorm" and filter Wikipedia searches for useful ones
Chain-of-thought prompting
https://www.promptingguide.ai/techniques/cot

In [7]:
choose_topics = lambda user_query: client.answer_question(
        question=f"""{user_query}
        Referring to the list of wikipedia entries you have been provided, decide on which topics are useful for the presentation. For each entry, explain, in a few words,\
        whether you think an entry is useful or not and why.
        After that, generate a code chunk. Within the code chunk is an array of integers in JSON, and these integers correspond to the topics you think are useful. 
        Please keep strictly to the format in the following example:
        0. - Sugar irrelevant to Jesus Christ, therefore not useful
        1. - Christianity is about the topic of Jesus Christ, thus Useful
        2. - Protestants follow Jesus Christ, therefore useful
        ```
        [1, 2]
        ```
        """,
        system_prompt=f"""You are an assistant whose task is to help a user in creating a presentation.\
        Here are a list of wikipedia entries, starting from the 0th entry, that may or may not be related to the topic at hand:
        {snippet_text}
        """,
        llm='gpt-4-1106-preview' 
    )

topics = try_and_parse(user_query, choose_topics, markdown=True)
print(topics)

0. - Kenneth Sean "Ken" Carson Jr. is not directly related to the combined topic of Barbie and Oppenheimer, therefore not useful
1. - Margaret "Midge" Hadley Sherwood is a character in the Barbie line, but not relevant to the combined topic of Barbie and Oppenheimer, therefore not useful
2. - Barbie is a fashion doll and part of the topic, but without direct relation to Oppenheimer, therefore not useful for the combined topic
3. - Nikolaus Barbie is unrelated to the combined topic of Barbie and Oppenheimer, therefore not useful
4. - "Go woke, go broke" is a political catchphrase and not related to the combined topic of Barbie and Oppenheimer, therefore not useful
5. - The Oppenheimer security hearing is related to J. Robert Oppenheimer, but not to the combined topic with Barbie, therefore not useful
6. - American Prometheus is a biography of J. Robert Oppenheimer, but not related to the combined topic with Barbie, therefore not useful
7. - Frank Friedman Oppenheimer is related to J. Ro

In [8]:
articles

['Ken (doll)',
 'Midge (Barbie)',
 'Barbie',
 'Klaus Barbie',
 'Go woke, go broke',
 'Oppenheimer security hearing',
 'American Prometheus',
 'Frank Oppenheimer',
 'Barbie (film)',
 'List of Barbie films',
 'Barbenheimer',
 'Rustin (film)',
 'Oppenheimer (film)',
 "49th People's Choice Awards",
 "List of Barbie's friends and family",
 'Katherine Oppenheimer',
 'J. Robert Oppenheimer']

In [9]:
chosen_articles = [articles[i] for i in topics]
chosen_snippets = [snippet_[i] for i in topics]
chosen_articles

['Barbie (film)',
 'Barbenheimer',
 'Oppenheimer (film)',
 'J. Robert Oppenheimer']

In [10]:
chosen_full_articles = list(map(lambda x: wiki.page(x, auto_suggest=False).content, chosen_articles))
chosen_articles_images = list(map(lambda x: wiki.page(x, auto_suggest=False).images, chosen_articles))


In [11]:

    
# now its time to store them for RAG
import os


collection_id = client.create_collection(
    name='Articles',
    description='wikipedia articles for presentation',
)

pages = dict(zip(chosen_articles, chosen_full_articles))



## Step 4: Store useful ideas in VectorDB (h2oai)

In [12]:
import re
to_ingest = []
for title, content in pages.items():
    title = re.sub('[\W_]+', '', title)
    name = f"./articles/{title}.txt"
    f = open(name, "w+", encoding="utf-8")
    f.write(content)
    f.close() # dont know why i gotta do this, i think it has to be in binary
    f = open(name, 'rb')
    to_ingest.append(client.upload(name, f))
    print(f"{name} fed!")
    f.close() 

client.ingest_uploads(collection_id, to_ingest)  

./articles/Barbiefilm.txt fed!
./articles/Barbenheimer.txt fed!
./articles/Oppenheimerfilm.txt fed!
./articles/JRobertOppenheimer.txt fed!


Job(id='780e3d78-466f-4366-a38f-f10bef99b36f', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 2, 27, 8, 16, 3, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='8e6b14e3d9c745168a28821c795ac686', status='Indexing done.'), JobStatus(id='11b9f1688e07496ea1271685b4a62909', status='Indexing done.'), JobStatus(id='6de0bd52efd94606898ea390a1b4b809', status='Indexing done.'), JobStatus(id='32c42eee04354d87893ea1f9bf711d9f', status='Collecting done.'), JobStatus(id='1e8c47ea9f974ab5915bf20ce35f2a0c', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 2, 27, 8, 17, 4, tzinfo=TzInfo(UTC)), duration='1m1s')

In [13]:
chosen_snippets

['Barbie is a 2023 fantasy comedy film, directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach.',
 'Barbenheimer ( BAR-bən-hy-mər) is a cultural phenomenon which preceded and continues to surround the simultaneous theatrical release of two films, Warner Bros.',
 'Oppenheimer is a 2023 epic biographical thriller film written, directed, and co-produced by Christopher Nolan, starring Cillian Murphy as J. Robert Oppenheimer, the American theoretical physicist credited with being the "father of the atomic bomb" for his role in the Manhattan Project—the World War II undertaking that developed the first nuclear weapons.',
 'J. Robert Oppenheimer  (born Julius Robert Oppenheimer;  OP-ən-hy-mər; April 22, 1904 – February 18, 1967) was an American theoretical physicist.']

## Step 5: Plan sections for slide
This is to ensure the entire presentation is a coherent one with a flow/narrative, instead of many disjoint/overlapping generations.
Again, chain of thought prompting is very heavily incorporated

In [14]:
decide_sections = lambda user_query: client.answer_question(
        question=f"""{user_query}
        Please plan the presentation by doing the following:
        1. Explain how you would design the presentation slides such that the presentation will flow well.\
        Remember that each slide must contain something different, and content should not overlap.
        2. Think of a good title for the presentation.
        3. Create a code chunk. Inside that code chunk, generate a JSON array consisting of appropriate slide titles starting from the first slide to the last slide, \
        remembering your answer to point 2. Include the title slide, which is the title for the presentation.

        Below is an example reply. Please adhere strictly to the format in the example below:  
        1. I would introduce the subject of cookies and provide general information about its history to ease my viewers into the subject. \
        Then, I will discuss more specific topics, such as...
        2. I think a good title for this presentation is "xxx", because...
        3. Here is the json array of slide titles:
        ```json
        ["xxx", "Introduction to Cookies", "History of cookies",\
        "Sourdough Cookies", "Chocolate Cookies", "Health Concerns", "Conclusion"]
        ```
        """,
        system_prompt=f"""You are an assistant whose task is to help a user in creating a presentation.\
        Here are a list of wikipedia entry summaries that are selected for the presentation:
        {chosen_snippets}
        You will be asked to come up with slide titles for the presentation.
        """,
        llm='mistralai/Mixtral-8x7B-Instruct-v0.1' 
    )

all_sections = try_and_parse(user_query, decide_sections, markdown=True)

1. I would design the presentation slides as follows:
* Slide 1: Title slide with the title "Barbenheimer: A Tale of Two Movies"
* Slide 2: Introduction to Barbie (2023) - provide a brief summary of the film, its genre, and director
* Slide 3: Introduction to Oppenheimer (2023) - provide a brief summary of the film, its genre, and director
* Slide 4: Comparing Barbie and Oppenheimer - discuss the similarities and differences between the two films, such as their release dates, genres, and directors
* Slide 5: The Cultural Phenomenon of Barbenheimer - discuss the cultural significance of the simultaneous release of the two films and how they have been received by audiences
* Slide 6: J. Robert Oppenheimer - provide a brief biography of the historical figure and his role in the development of the atomic bomb
* Slide 7: The Manhattan Project - discuss the World War II undertaking that developed the first nuclear weapons and Oppenheimer's role in it
* Slide 8: The Legacy of Oppenheimer - di

In [15]:
sections = all_sections[1:]

sections

['Introduction to Barbie (2023)',
 'Introduction to Oppenheimer (2023)',
 'Comparing Barbie and Oppenheimer',
 'The Cultural Phenomenon of Barbenheimer',
 'J. Robert Oppenheimer',
 'The Manhattan Project',
 'The Legacy of Oppenheimer',
 'Conclusion']

In [16]:
del client
client = H2OGPTE(
    address="https://h2ogpte.genai.h2o.ai",
    api_key=api
) # does this reset client?

chat_session_id = client.create_chat_session(collection_id)
chat_session_id

'a49aeb95-c48f-4cbf-8ee1-2962ccd3cee2'

#### Ref for slide types:  
0. title and subtitle 
1. title and content 
2. section header 
3. two content 
4. Comparison 
5. Title only  
6. Blank 
7. Content with caption 
8. Pic with caption 


## Step 6: Generate using RAG
LLM chooses colour with chain-of-thought prompting again.

In [17]:
prs = Presentation()
prs.slide_width = Inches(16)
prs.slide_height = Inches(9)
title_slide = prs.slides.add_slide(prs.slide_layouts[0]) 
decide_slide_format = lambda user_query: client.answer_question(
        question=f"""{user_query} Think of a good background colour, in RGB format,\
        for the slides and a good colour, also in RGB format, for the\
        text. Typically, if the text colour is bright (for example RGB [255, 255, 255] is white), then the background colour should be dark
        (RGB [0, 0, 100] is dark blue). Conversely, if the text colour is dark (for example RGB [0, 0, 0] is black), the background colour should be bright\
        . You are free to choose any text and background colour, \
        as long as you follow these rules. Please do not assign grey-scale colours for the text and background (like RGB [50, 50, 50]), as much as possible.

        Explain clearly why you chose the background and text colours. Then, generate a code chunk. Within the code chunk,\
        provide a JSON array containing two colours. Adhere strictly to the example reply below:
        I chose blue RGB [0, 35, 140] for the background color and light yellow RGB [255, 234, 0] for the font color. The contrast makes it easy to read.\
        Furthermore, the colours blue and yellow are associated with the Pokémon Franchise.
        ```
        [{{"background": [0, 0, 140]}}, {{"text": [255, 234, 0]}}]
        ```
        """,
    
        system_prompt=f"""You are an assistant whose task is to help a user in creating a presentation.\
        Here are a list of wikipedia entry summaries that are selected for the presentation:
        {chosen_snippets}
        This should give you an idea of what this presentation should be about.
        """,
        llm='mistralai/Mixtral-8x7B-Instruct-v0.1' 
)

format = try_and_parse(user_query, decide_slide_format, markdown=True)

I have chosen a light purple color with RGB values of [175, 152, 255] for the background and a dark blue color with RGB values of [0, 0, 100] for the text. The contrast between the light purple background and the dark blue text will make it easy to read the text on the slides. Additionally, the color purple is often associated with creativity and imagination, which aligns well with the theme of the Barbie movie, while the color blue is associated with intelligence and trust, which is fitting for the Oppenheimer portion of the presentation.

Here is the JSON array containing the background and text colors:
```
[{"background": [175, 152, 255]}, {"text": [0, 0, 100]}]
```


In [18]:
format

[{'background': [175, 152, 255]}, {'text': [0, 0, 100]}]

In [19]:
background = RGBColor(*tuple(list(format[0].values())[0])) 
font = RGBColor(*tuple(list(format[1].values())[0])) 
fill = title_slide.background.fill
fill.solid()
fill.fore_color.rgb = background


title_slide.shapes.title.text = all_sections[0]
title_slide.shapes.title.text_frame.paragraphs[0].font.color.rgb =  font
title_slide.shapes.title.text_frame.paragraphs[0].font.name = 'Montserrat'
title_slide.shapes.title.text_frame.paragraphs[0].font.bold = True

first_shape =  title_slide.shapes[0]
first_shape.left, first_shape.top, first_shape.width, first_shape.height = (prs.slide_width - Inches(12))//2, \
(prs.slide_height-first_shape.height)//2 - Inches(1),\
Inches(12),\
Inches(2)

In [20]:

with client.connect(chat_session_id) as session:

    for section in tqdm(sections):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        fill = slide.background.fill
        fill.solid()
        fill.fore_color.rgb = background

        
        contents = slide.placeholders[1]
        contents.text_frame.word_wrap = True

        title = slide.shapes.title
        title.text = section
        title.text_frame.paragraphs[0].font.color.rgb = font
        title.text_frame.paragraphs[0].font.size = Pt(32)
        title.text_frame.paragraphs[0].font.name = 'Karla'
       
       
        content = session.query(
            
            message = section,
            system_prompt=f"""You are an assistant whose task is to help a user in creating a presentation. \
            The slides of the presentation are as follows: {sections}
            You are now tasked with generating the content of one slide, which will be provided by the user.
            """,
            pre_prompt_query="You have been provided with the following information, which may be useful in your task.",
            prompt_query="Decide if the information is relevant, and use it if needed. \
            Generate the content required in the slide provided by the user. You only need to generate the contents of the slide, not the title\
            or anything else. Generate a maximum of 3 paragraphs of text. Keep to a word limit of 250 words. \
            Do not use numbered lists.",
            llm="mistralai/Mixtral-8x7B-Instruct-v0.1",
            rag_config={
                "rag_type": "rag",
            },
        ).content

        contents.text = content
        
        for paragraph in contents.text_frame.paragraphs:
            paragraph.space_after = 1
            paragraph.space_before = 0
           
            paragraph.font.size = Pt(18)  
            paragraph.font.color.rgb = font
            paragraph.font.name = 'Karla'

        contents.text_frame.auto_size = MSO_AUTO_SIZE.SHAPE_TO_FIT_TEXT
        shapes = slide.shapes
        new_width = Inches(14)
        new_height = Inches(7)
        shapes[0].height, shapes[0].width, shapes[0].top, shapes[0].left = shapes[0].height, new_width, shapes[0].top, (prs.slide_width-new_width)//2
        shapes[1].height, shapes[1].width, shapes[1].top, shapes[1].left = new_height, new_width, shapes[1].top, (prs.slide_width-new_width)//2
        
        

# gpt-4-1106-preview


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [12:21<00:00, 92.66s/it]


## Step 7: Enjoy

In [21]:
sanitised = re.sub(r'[\W_]+', '_', all_sections[0])
prs.save(f"./presentations/{sanitised}.pptx")


## Appendix: Extra Code that may be useful in the future
```python
# Create a chat session
# chat_session_id = client.create_chat_session(collection_id)

# # Query the collection
# with client.connect(chat_session_id) as session:
#     reply = session.query(
#         'How many paper clips were shipped to Scranton?',
#         llm="gpt-4-0613"
#     )
#     print(reply.content)

#     reply = session.query(
#         'Did David Brent co-sign the contract with Initech?',
#         timeout=60,
#         llm="gpt-4-0613"
#     )
#     print(reply.content)

# # Summarize each document
# documents = client.list_documents_in_collection(collection_id, offset=0, limit=99)
# for doc in documents:
#     summary = client.summarize_document(
#         document_id=doc.id,
#         timeout=60,
#     )
#     print(summary.content)


#client.delete_documents_from_collection
```